# Create organizations & machines

The goal of this notebook is to demonstrate how to use InUse api in order to automate creation, modifications & deletions of organizations & machines.

Before going further, here's a quick reminder. Organizations are modeled using a 5 level hierarchy as follow:
- `Manufacturer`
  - `Producer `
    - `Site`
      - `Line`
        - `Machine`
        
Please note that a `Machine` can have other machines as children, which make the hierarchy potentially limitless in terms of height. With this mechanism, it's possibe to represent machine, components, subcomponents etc.

## Imports

In [ ]:
# Library used to type a password in a secured way.
import getpass
# Client library for the InUse API.
from pyinuse import InUse
# Library for logging
import logging
logging.basicConfig(format="%(asctime)s, %(name)s, %(levelname)s, %(message)s", level=logging.INFO)

## Create a client

First of all, we will create a client and login with it.

In [ ]:
inuse = InUse()
inuse.login(input("Username? "), getpass.getpass("Password? "))

Great, you're now authentified!

## Create a hierarchy

Let's find the manufacturer. Since the `Manufacturer` is the root object of the organization, listing them will return a list of one item only.

In [ ]:
manufacturer = inuse.manufacturers.list()[0]

Then, we will create a machine model named `Notebook machine model`.

In [ ]:
machine_model = inuse.machine_models.update_or_create(
    params={"manufacturer": manufacturer["pk"], "code": "notebook-machine-model"},
    data={
        "manufacturer": manufacturer["pk"],
        "code": "notebook-machine-model",
        "name": "Notebook machine model",
    },
)

Then, we will create a hierarchy composed of only one item per level: one producer, one site, one line & one machine. Obviously, you can create as many as you for each level.

In [ ]:
hierarchy = [
    {
        "name": "Notebook producer",
        "code": "notebook-producer",
        "type": "producer",
        "children": [
            {
                "code": "notebook-site-1",
                "name": "Notebook site 1",
                "type": "site",
                "children": [
                    {
                        "type": "line",
                        "code": "notebook-line-1",
                        "name": "Notebook line 1",
                        "children": [
                            {
                                "type": "machine",
                                "code": "notebook-machine-1",
                                "name": "Notebook machine 1",
                                "machine_model": machine_model["pk"],
                                "visible_on_share": True,
                            }
                        ],
                    }
                ],
            }
        ],
    },
]


def run(parent, parent_type, obj):
    if isinstance(obj, list):
        for item in obj:
            run(parent, parent_type, item)
    else:
        typ = obj.pop("type")
        api = getattr(inuse, f"{typ}s")
        ret = api.update_or_create(
            params={parent_type: parent["pk"], "code": obj["code"]},
            data={**obj, parent_type: parent["pk"]},
        )
        obj.update(ret)
        run(obj, typ, obj.get("children", []))


run(manufacturer, "manufacturer", hierarchy)

## Logout

This is important to logout once it's done.

In [ ]:
inuse.logout()